In [1]:
import os
import sys
import torch
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams['figure.figsize'] = [14, 8]
import motmetrics as mm
from motmetrics.distances import iou_matrix
import pandas as pd

sys.path.append('../..')
from pytracking.utils.load_text import load_text
from pytracking.evaluation import Tracker, get_dataset, trackerlist
from pprint import pprint

dd = get_dataset('got10k_probio_valid')

In [ ]:
names = list([i.name for i in dd])

In [49]:
import json
from collections import defaultdict
pt = '/mnt/data/qizhezhang/allDataset/ProBio/'
occ_all = dict()
occ = {'Less occlusion&no occlusion':1, 'Partial occlusion':2, 'Severe occlusion':3}
for i in os.listdir(pt):
    with open(os.path.join(pt, i, 'label.json')) as f:
        data = json.load(f)
    occ_obj = defaultdict(list)
    for frame in data:
        for item in frame['items']:
            try: 
                occ_obj[f'{i}_{255-int(item["object_id"])}'].append(occ[item['occlusion']])
            except: pass
    occ_all.update(occ_obj)

In [50]:
import numpy as np
def count_occ(obj):
    obj = np.array(obj)
    # 统计obj中1,2,3出现的次数
    return [np.sum(obj==1), np.sum(obj==2), np.sum(obj==3)]

for i in occ_all:
    occ_all[i] = count_occ(occ_all[i])
occ_all

{'0802_RGB_0_0_253': [3, 0, 120],
 '0802_RGB_0_0_245': [91, 30, 2],
 '0802_RGB_0_0_247': [118, 5, 0],
 '0802_RGB_0_0_250': [29, 22, 70],
 '0802_RGB_0_0_246': [90, 32, 1],
 '0802_RGB_0_0_252': [1, 2, 120],
 '0802_RGB_0_0_248': [122, 0, 1],
 '0802_RGB_0_0_254': [119, 4, 0],
 '0802_RGB_0_0_249': [122, 1, 0],
 '0802_RGB_0_0_251': [2, 2, 118],
 '0802_RGB_0_0_244': [27, 52, 40],
 '0802_RGB_0_1_253': [1, 1, 24],
 '0802_RGB_0_1_249': [27, 21, 2],
 '0802_RGB_0_1_251': [0, 34, 17],
 '0802_RGB_0_1_252': [0, 3, 48],
 '0802_RGB_0_1_248': [49, 1, 1],
 '0802_RGB_0_1_254': [51, 0, 0],
 '0802_RGB_0_1_250': [17, 1, 11],
 '0802_RGB_0_1_247': [48, 1, 2],
 '0802_RGB_0_1_246': [21, 16, 11],
 '0802_RGB_0_1_245': [18, 3, 20],
 '0802_RGB_0_1_244': [39, 4, 0],
 '0802_RGB_0_2_251': [1, 152, 0],
 '0802_RGB_0_2_244': [121, 4, 3],
 '0802_RGB_0_2_252': [1, 152, 0],
 '0802_RGB_0_2_245': [106, 4, 5],
 '0802_RGB_0_2_253': [0, 151, 0],
 '0802_RGB_0_2_250': [0, 153, 0],
 '0802_RGB_0_2_247': [153, 0, 0],
 '0802_RGB_0_2_24

In [ ]:
for i in occ_all:
    with open('/mnt/data/qizhezhang/allDataset/GOT-10k/probio/'+i+'groundtruth.txt', 'r') as f:
        lines = np.array(f.readlines())
        